In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import files
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(19,)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
file = 'gdrive/My Drive/train.csv'
df=pd.read_csv(file)

df['Deck'] = df['Cabin'].str[0]

df = df.drop(columns=['Cabin'], axis=1)


df = df.drop(columns=['Name', 'Ticket'], axis=1)


df = df.fillna(df.mean())

#Categorias a serem codificadas para valores binários
categorical_cols = ['Sex', 'Embarked', 'Deck']

#Codificação
df_encoded = pd.get_dummies(df, columns=categorical_cols)

X = df_encoded.drop(columns=['Survived'], axis=1)
print(len(X.columns))
Y = df_encoded['Survived']


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
history = model.fit(X_train_scaled, y_train, epochs=40, batch_size=32, validation_split=0.2, callbacks=[checkpoint])
best_model = tf.keras.models.load_model('best_model.h5')

test_loss, test_acc = best_model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

19
Epoch 1/40


<ipython-input-4-a57cfcfe65c0>:12: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


18/18 [==============================] - 6s 106ms/step - loss: 0.6454 - accuracy: 0.6221 - val_loss: 0.4608 - val_accuracy: 0.8531
Epoch 2/40
 1/18 [>.............................] - ETA: 0s - loss: 0.4819 - accuracy: 0.8438

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


18/18 [==============================] - 1s 56ms/step - loss: 0.5203 - accuracy: 0.7592 - val_loss: 0.5433 - val_accuracy: 0.8112
Epoch 3/40
18/18 [==============================] - 2s 90ms/step - loss: 0.4492 - accuracy: 0.8155 - val_loss: 0.4557 - val_accuracy: 0.8462
Epoch 4/40
18/18 [==============================] - 1s 54ms/step - loss: 0.4280 - accuracy: 0.8260 - val_loss: 0.4930 - val_accuracy: 0.8531
Epoch 5/40
18/18 [==============================] - 1s 64ms/step - loss: 0.3964 - accuracy: 0.8330 - val_loss: 0.4075 - val_accuracy: 0.8531
Epoch 6/40
18/18 [==============================] - 1s 53ms/step - loss: 0.3956 - accuracy: 0.8401 - val_loss: 0.5235 - val_accuracy: 0.8741
Epoch 7/40
18/18 [==============================] - 1s 62ms/step - loss: 0.4341 - accuracy: 0.8348 - val_loss: 0.3703 - val_accuracy: 0.8601
Epoch 8/40
18/18 [==============================] - 1s 54ms/step - loss: 0.4297 - accuracy: 0.8102 - val_loss: 0.5846 - val_accuracy: 0.8112
Epoch 9/40
18/18 [======

In [5]:
testFile = 'gdrive/My Drive/test.csv'
df_test = pd.read_csv(testFile)

df_test['Deck'] = df_test['Cabin'].str[0]
df_test = df_test.drop(columns=['Cabin', 'Name', 'Ticket'], axis=1)
df_test = df_test.fillna(df_test.mean())

df_test['Deck_T'] = 0

train_deck_categories = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

for category in train_deck_categories:
    if category not in df_test['Deck'].unique():
        df_test[category] = 0

df_test_encoded = pd.get_dummies(df_test, columns=['Sex', 'Embarked', 'Deck'])

encoded_columns = df_test_encoded.columns.tolist()

if 'Deck_T' in encoded_columns:
    encoded_columns.remove('Deck_T')


encoded_columns.append('Deck_T')

df_test_encoded = df_test_encoded[encoded_columns]


X_test_final = df_test_encoded
PassengerId = df_test_encoded['PassengerId']


scaler = StandardScaler()
X_test_scaled_final = scaler.fit_transform(X_test_final)


best_model = tf.keras.models.load_model('best_model.h5')


predictions = best_model.predict(X_test_scaled_final)


binary_predictions = np.round(predictions).astype(int)

output_df = pd.DataFrame({'PassengerId': PassengerId, 'Survived': binary_predictions.flatten()})
output_df.to_csv('predictions.csv', index=False)

#files.download('predictions.csv')

<ipython-input-5-06af6259b2ea>:6: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_test = df_test.fillna(df_test.mean())


14/14 [==============================] - 0s 10ms/step
